<a href="https://colab.research.google.com/github/alibekk93/soccer_fun_stuff/blob/main/FBRef_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FBRef data getter
https://fbref.com/en/

## Setup

In [1]:
!pip install requests_html

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.8 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=fa852508777397b4b9b1af2da1179d5b3addcbcad89374abe80129ceeb150699
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.6
    Uninstalling urllib3-2.0.6:
      Successfully uninstalled urllib3-2.0.6


In [82]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import requests
from requests_html import HTMLSession
from bs4 import BeautifulSoup

In [91]:
session = HTMLSession()

## Summary Tables

In [95]:
def get_team_summary_table(tables:list, table_index:int=0) -> pd.DataFrame():
  """Parses team summary table given a list of all tables from URL
  """
  table = tables[table_index]

  all_headers = [th.text for th in table.find_all('th')]
  data_headers_s = all_headers.index('Nation')
  data_headers_e = all_headers.index('Matches')
  table_headers = all_headers[data_headers_s:data_headers_e]
  table_headers[-10:] = ['p90_' + th for th in table_headers[-10:]]
  player_names = all_headers[data_headers_e+1:-2]
  table_data = [th.text for th in table.find_all('td')]
  table_data_cleaned = [r.tolist()[:-1] for r in np.array(table_data).reshape(-1, 33)][:-2]

  summary_df = pd.DataFrame(table_data_cleaned, columns=table_headers)
  summary_df['Player'] = player_names

  summary_df = summary_df[summary_df['Min']!='']

  for col in summary_df.columns:
    try:
      summary_df.loc[:, col] = summary_df.loc[:, col].astype(float)
    except:
      pass

  return summary_df

In [ ]:
team_urls =   ['https://fbref.com/en/squads/b8fd03ef/2022-2023/Manchester-City-Stats',
               'https://fbref.com/en/squads/18bb7c10/2022-2023/Arsenal-Stats',
               'https://fbref.com/en/squads/19538871/2022-2023/Manchester-United-Stats',
               'https://fbref.com/en/squads/b2b47a98/2022-2023/Newcastle-United-Stats',
               'https://fbref.com/en/squads/822bd0ba/2022-2023/Liverpool-Stats',
               'https://fbref.com/en/squads/d07537b9/2022-2023/Brighton-and-Hove-Albion-Stats',
               'https://fbref.com/en/squads/8602292d/2022-2023/Aston-Villa-Stats',
               'https://fbref.com/en/squads/361ca564/2022-2023/Tottenham-Hotspur-Stats',
               'https://fbref.com/en/squads/cd051869/2022-2023/Brentford-Stats',
               'https://fbref.com/en/squads/fd962109/2022-2023/Fulham-Stats',
               'https://fbref.com/en/squads/47c64c55/2022-2023/Crystal-Palace-Stats',
               'https://fbref.com/en/squads/cff3d9bb/2022-2023/Chelsea-Stats',
               'https://fbref.com/en/squads/8cec06e1/2022-2023/Wolverhampton-Wanderers-Stats',
               'https://fbref.com/en/squads/7c21e445/2022-2023/West-Ham-United-Stats',
               'https://fbref.com/en/squads/4ba7cbea/2022-2023/Bournemouth-Stats',
               'https://fbref.com/en/squads/e4a775cb/2022-2023/Nottingham-Forest-Stats',
               'https://fbref.com/en/squads/d3fd31cc/2022-2023/Everton-Stats',
               'https://fbref.com/en/squads/a2d435b3/2022-2023/Leicester-City-Stats',
               'https://fbref.com/en/squads/5bfb9659/2022-2023/Leeds-United-Stats',
               'https://fbref.com/en/squads/33c895d4/2022-2023/Southampton-Stats
               ',]

dfs = []

for url in tqdm(team_urls):
  r = session.get(url)
  team = url.split('/')[-1][:-6]
  soup = BeautifulSoup(r.html.raw_html)
  tables = soup.find_all('table')
  team_summary = get_team_summary_table(tables)
  team_summary['Team'] = team
  dfs.append(team_summary)

summary_data = pd.concat(dfs)

In [114]:
summary_data.to_csv('PL_2022_player_summary.csv', index=False)

In [120]:
summary_data.groupby('Nation').sum()['Gls'].sort_values(ascending=False)

<ipython-input-120-467e3d3e50bb>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  summary_data.groupby('Nation').sum()['Gls'].sort_values(ascending=False)


Nation
eng ENG    349.0
br BRA      97.0
no NOR      52.0
pt POR      49.0
ar ARG      36.0
           ...  
ba BIH       0.0
ve VEN       0.0
au AUS       0.0
at AUT       0.0
zw ZIM       0.0
Name: Gls, Length: 61, dtype: float64